<a href="https://colab.research.google.com/github/Ahirvoas/Training-Day1/blob/main/Tutorial_day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install condacolab &> /dev/null
import condacolab
condacolab.install()
!conda update -n base -c conda-forge conda &> /dev/null

✨🍰✨ Everything looks OK!


### Install the python toolbox package

In [ ]:
import os, sys

if os.path.isdir('python-toolbox'):
  !rm -rf python-toolbox
!git clone http://github.com/OpenFAST/python-toolbox
%cd python-toolbox
!python -m  pip install -e . &> /dev/null
sys.path.append(os.getcwd())
%cd ..

Cloning into 'python-toolbox'...
remote: Enumerating objects: 3011, done.
remote: Counting objects: 100% (1230/1230), done.
remote: Compressing objects: 100% (566/566), done.
remote: Total 3011 (delta 731), reused 997 (delta 651), pack-reused 1781
Receiving objects: 100% (3011/3011), 2.49 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (1829/1829), done.
/content/python-toolbox
/content


In [ ]:
!conda install -c conda-forge openfast &> /dev/null

### Download the folder

In [ ]:
!rm -rf Training-Day1
!git clone https://github.com/Ahirvoas/Training-Day1

Cloning into 'Training-Day1'...
remote: Enumerating objects: 1898, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 1898 (delta 22), reused 44 (delta 13), pack-reused 1843
Receiving objects: 100% (1898/1898), 119.12 MiB | 24.41 MiB/s, done.
Resolving deltas: 100% (1705/1705), done.


# Wind Turbine Modeling Workshop

## OpenFAST

OpenFAST is a multi-physics and multi-fidelity tool for the simulation of the coupled dynamic response of wind turbines. The software allows the analysis of a wide range of wind turbine configurations, including those with two- or three-bladed horizontal axis rotors, pitch regulated or fixed pitch, rigid or tilting hubs, downwind or upwind rotors, and lattice or tubular towers. In addition, the wind turbine can be modelled on land or at sea on fixed or floating substructures.

<img src="https://drive.google.com/uc?export=view&id=1UqbFPXcbv035NGfpkbq8thaMF3An8PaT" width="500"/>

## OpenFAST - python toolbox

pyfast is a Python package developed by the NREL team to couple quickly and easily the OpenFAST tool. This package allows the modification and post-processing of files.

In [ ]:
# Package pyfast importation  (and some useful packages)

from pyFAST.case_generation.runner import run_fast
import pyFAST.postpro as postpro
from pyFAST.input_output import FASTOutputFile
from pyFAST.input_output import FASTInputFile
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import os

### Case - Floating offfshore wind turbine steady wind

<img src="https://drive.google.com/uc?export=view&id=1vddiFGtyuvC0u7X3GUwbSrYkZbmYdT5B" width="250"/>

This tutorial is based on a **offshore reference wind turbine** developed within the Offshore Code Comparison 3 project which is a spar buoy structure with the NREL 5MW wind turbine atop.The gross properties chosen for the NREL 5-MW baseline wind turbine are gathered in the following table.

| Rating | 5 MW |
|:---:|:---:|
| Rotor orientation, Configuraiton | Upwind, 3 Blades |
| Rotor, Hub Diameter | 126 m, 3m |
| Hub Height | 90 m |
| Cut-In, Rated, Cut-Out Wind Speed | 3m/s, 11.4m/s, 25m/s |
| Cut-in, rated rotor speed | 6.9 rpm, 12.1 rpm |
| Rotor Mass | 110,000 kg |
| Nacelle Mass | 240,000 kg |
| Tower Mass | 347,460 kg |

In this tutorial, we will consider, hereafter, a **steady wind** as external sollicitation.

In [ ]:
path = os.path.join(os.getcwd(), 'Training-Day1', 'OffshoreSteadyWindCtrl')

In [ ]:
# Specify the OpenFAST binaries and OpenFAST main file for simulation

FAST_EXE  = r'/usr/local/bin/openfast' # Location of a FAST exe (and dll) (FAST_EXE  = openfast)
main_file = os.path.join(path, 'NRELOffshrBsline5MW.fst')  # Main file in OpenFAST directory

### 1 - Run your first wind turbine simulation

#### Run your first simulation thanks to OpenFAST software. We will use the *run_fast* function developed in the pyFAST package.

In [ ]:
# Run OpenFAST

print('Running example script: {}'.format(main_file))
run_fast(main_file, fastExe=FAST_EXE, wait=True, showOutputs=True, showCommand=True)

Running example script: /content/Training-Day1/OffshoreSteadyWindCtrl/NRELOffshrBsline5MW.fst
Running: /usr/local/bin/openfast /content/Training-Day1/OffshoreSteadyWindCtrl/NRELOffshrBsline5MW.fst


<pyFAST.case_generation.runner.run_cmd.<locals>.Dummy at 0x7f25c47fba60>

#### Look a the OpenFAST working directory and find the modification (i.e., the new files created after the OpenFAST run).

In [ ]:
# Average Post-treatment of OpenFAST output file

outFile = os.path.join(path, 'NRELOffshrBsline5MW.out') # path\outputfile
avg_results = postpro.averagePostPro([outFile])
print('Average results on full simulation length')
print(avg_results)

Average results on full simulation length
   Time_[s]  Wind1VelX_[m/s]  Wind1VelY_[m/s]  Wind1VelZ_[m/s]  \
0  32.55625              8.0              0.0              0.0   

   BldPitch1_[deg]  Azimuth_[deg]  RotSpeed_[rpm]  GenSpeed_[rpm]  \
0              0.0     180.381555        8.750559      848.781155   

   NacYaw_[deg]  OoPDefl1_[m]  ...  B1RdtFzi_[N]  B1RdtMxi_[N-m]  \
0      0.000938      2.976672  ...   -897.430432    10246.392894   

   B1RdtMyi_[N-m]  B1RdtMzi_[N-m]       T[1]_[N]     T_a[1]_[N]      T[2]_[N]  \
0   -3.415471e+06   -7.343369e-11  703462.426624  529303.228813  1.066448e+06   

      T_a[2]_[N]      T[3]_[N]     T_a[3]_[N]  
0  892337.975524  1.063370e+06  889271.114433  

[1 rows x 95 columns]


#### Read an OpenFAST output file and convert it to a pandas DataFrame

In [ ]:
df = FASTOutputFile(outFile).toDataFrame()

In [ ]:
df.head()

,Time_[s],Wind1VelX_[m/s],Wind1VelY_[m/s],Wind1VelZ_[m/s],BldPitch1_[deg],Azimuth_[deg],RotSpeed_[rpm],GenSpeed_[rpm],NacYaw_[deg],OoPDefl1_[m],...,B1RdtFzi_[N],B1RdtMxi_[N-m],B1RdtMyi_[N-m],B1RdtMzi_[N-m],T[1]_[N],T_a[1]_[N],T[2]_[N],T_a[2]_[N],T[3]_[N],T_a[3]_[N]
0,0.0000,8.0,-0.0,0.0,0.0,0.000000,8.000000,776.00000,0.000000,0.000000,...,-17496.096,6998.1641,95388768.0,-2.664432e-16,911089.50,736845.19,911089.94,736845.69,911089.94,736845.69
1,0.0125,8.0,-0.0,0.0,0.0,0.600155,8.004406,773.74615,-0.000039,0.004239,...,-17275.918,31066.5620,95910784.0,1.297027e-10,911088.88,736844.62,911090.31,736846.06,911090.31,736846.06
2,0.0250,8.0,-0.0,0.0,0.0,1.200638,8.007770,771.60535,-0.000141,0.016629,...,-17037.316,69358.6480,96330104.0,4.047802e-10,911087.06,736842.81,911091.44,736847.19,911091.44,736847.19
3,0.0375,8.0,-0.0,0.0,0.0,1.801257,8.008302,769.63898,-0.000278,0.036902,...,-16775.879,111398.9100,96656032.0,7.534835e-10,911084.06,736839.81,911093.38,736849.06,911093.31,736849.06
4,0.0500,8.0,-0.0,0.0,0.0,2.401822,8.006063,767.89429,-0.000416,0.065093,...,-16493.660,145158.4400,96907392.0,1.090087e-09,911079.88,736835.56,911096.00,736851.69,911096.00,736851.62


#### Exercice:

##### Rotor speed of the wind turbine:

Plot the rotor speed of the wind turbine versus the time obtained from the previous simulation. The variables of interest in the Pandas Dataframe are respectively named *RotSpeed_[rpm]* and *Time_[s]*.

Hint:

You can use the matplotlib library in order to plot y versus x as lines.

For information [click here](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html) and an example:
```python
import matplotlib.pyplot as plt

x = [1, 2, 3]
y = np.array([[1, 2], [3, 4], [5, 6]])
plot(x, y)
```

In [ ]:
# INSERT CODE

##### Rotor speed and generated power of the wind turbine:

Plot the rotor speed and the generated power of the wind turbine versus the time obtained from the previous simulation. For comparison, you can plot the two time-series on the same graph thanks to the function *subplot*.

Hint:

You can find the corresponding variables of the Pandas Dataframe for the electrical generator power by looking at the Excel file *OutListParameters.xlsx*.

An example of the python function to be used:

```python
import numpy as np
import matplotlib.pyplot as plt


def f(t):
    return np.exp(-t) * np.cos(2*np.pi*t)


t1 = np.arange(0.0, 5.0, 0.02)

plt.figure()
plt.subplot(211)
plt.plot(t1, f(t1), color='black')

plt.subplot(212)
plt.plot(t1, np.cos(2*np.pi*t1), color='tab:orange', linestyle='--')
plt.show()
```

In [ ]:
# INSERT CODE

### 2 - Modification of the OpenFAST files

In practice, OpenFAST models the system via computational modules for aerodynamics, hydrodynamics for offshore structures, control and electrical (servo) system dynamics, and structural dynamics to enable coupled non-linear aero-hydro-servo-elastic simulation in the time domain.

<img src="https://drive.google.com/uc?export=view&id=1dvyZ1GJtchd-H5oamzObL6XOaqFi0uN3" width="500"/>

Each of these modules is independent of the others and is based on a configuration file that creates an arborescence of files to carry out an OpenFAST simulation.

<img src="https://drive.google.com/uc?export=view&id=1jBPGzDmvX49-6FInYqm1KM9evxC_BJRw" width="500"/>

#### Modification of the wind speed magnitude

In this part, we will modify the wind speed magnitude to the rated value. For that purpose we will have to change a specific variable in the InflowWind file related to the wind-inflow external sollicitation.

First, you will have to open the InflowWind file and find the correct value to change. **Remember** that we consider in this tutorial for simplicity a steady wind solicitation.

**Hint:**

    Pay attention to the comments in the file of interest !

##### Modification of the InflowWind file

In [ ]:
filename_inflow = os.path.join(path, 'InflowWind.dat') # Insert the name of the InflowWind file with the path (relative or absolute)
f_inflow = FASTInputFile(filename_inflow)
f_inflow['HWindSpeed'] = 11.4 # Insert the rated speed of the 5MW wind turbine
new_filename_inflow = 'InflowWind_{}mps.dat'.format(int(f_inflow['HWindSpeed'])) # New name of the InflowWind file
save_path = os.path.join(path, new_filename_inflow)
f_inflow.write(save_path) # Save the new generated InflowWind file

##### Modification of the main OpenFAST file

In [ ]:
f_mainfile = FASTInputFile(main_file)
f_mainfile['InflowFile'] = '"'+new_filename_inflow+'"'
new_filename_mainfile = os.path.join(path, 'Main_{}mps.fst'.format(int(f_inflow['HWindSpeed'])))
f_mainfile.write(new_filename_mainfile)

##### Run OpenFAST with the new modified files.

**Hint:**
    
    You can use the previous command use to run the simulation

In [ ]:
# INSERT CODE

##### Rotor speed and generated power of the wind turbine:

Plot the rotor speed and the generated power of the wind turbine versus the time obtained from the previous simulation. For comparison, you can plot the two time-series on the same graph thanks to the function *subplot* as previously.

Please plot also the results of the 1-st simulation obtained in this tutorial for comparison!

**Hint:**

    Load the output file in a new pandas dataframe

In [ ]:
# INSERT CODE

In [ ]:
# INSERT CODE

#### Time for discussion